In [365]:
import random as rnd
import copy as cp


class Chromosome:
    def __init__(self, genes_no):
        self._pool = []
        self._generate_pool(genes_no)

        
    def _generate_pool(self, genes_no):
        for i in range(0, genes_no):
            self._pool.append(rnd.randint(0,1))
        
        
    def _get_fitness(self):
        ones = 0
        
        for i in range(0, len(self._pool)):
            if self._pool[i] == 1:
                ones += 1
                
        return ones
    
    
    def get_score(self):
        return self._get_fitness() / len(self._pool)
    
    
    def crossover_with(self, chromosome, points_no):
        if chromosome is None:
            raise AttributeError("The crossover chromosome is absent.")
        
        if points_no >= len(self._pool) or points_no <= 0:
            raise AttributeError("The number of crossover points is improper.")
            
        if len(chromosome._pool) != len(self._pool):
            raise AttributeError("The number of genes between chromosomes differs.")
            
        self._make_crossover(chromosome, points_no)
    
    
    def _get_partition_points(self, points_no):
        # Indices can be the same.
        indices = []
        
        for i in range(0, points_no):
            index = rnd.randint(0, len(self._pool)-1)
            indices.append(index)
            
        return indices
    
    
    def _make_crossover(self, chromosome, points_no):
        indices = self._get_partition_points(points_no)
        
        for i in range(0, len(indices)):
            crossover_index = indices[i]
            from_left = True
            
            if rnd.random() > 0.5:
                from_left = False
                
            #print('from_left=' + str(from_left) + ', crossover_index=' + str(crossover_index))
            #print('lhs=' + str(self._pool) + ',rhs=' + str(chromosome._pool) + ' -> ', end='')
            self._exchange_genes(crossover_index, chromosome, i, from_left)
            #print('lhs=' + str(self._pool) + ',rhs=' + str(chromosome._pool) + '.', end='\n')
    
    
    def _exchange_genes(self, crossover_index, chromosome, j, from_left):
        if from_left == True:    
            for j in range(0, crossover_index):
                self._swap_genes(chromosome, j)
            
            return
        
        for k in range(crossover_index, len(self._pool)):
            self._swap_genes(chromosome, k)

        
    def _swap_genes(self, chromosome, j):
        tmp = self._pool[j]
        self._pool[j] = chromosome._pool[j]
        chromosome._pool[j] = tmp
        
        
    def mutate(self, points_no):
        if points_no > len(self._pool):
            raise AttributeError("Cannot mutate more genes then available.")
        
        indices = self._get_partition_points(points_no)
        #print('indices=' + str(indices) + ', pool=' + str(self._pool), end='')
        
        for i in range(0, len(indices)):
            self._flip_gene(indices[i])
        
        #print(' -> pool=' + str(self._pool))
            
            
    def _flip_gene(self, index):
        if self._pool[index] == 1:
            self._pool[index] = 0
            return
        
        self._pool[index] = 1
        
        
    def to_string(self):
        score = str(int(self.get_score() * 100)) + '%'
        return '[' + str(self._pool) + ',' + score + ']'
    
    
    def get_copy(self):
        chromo = Chromosome(len(self._pool))
        chromo._pool = cp.deepcopy(self._pool)
        return chromo
    
    
    def size(self):
        return len(self._pool)
    

In [162]:
#Chromosome demo
GENES_NO = 6
chromo = Chromosome(GENES_NO)

print('pool=' + str(chromo._pool))
print('fitness=' + str(chromo._get_fitness()))
print('score=' + str(chromo.get_score()))
print()
print('CROSSOVER')
print(chromo.crossover_with(Chromosome(GENES_NO), 3))
print()
print('MUTATION')
print(chromo.mutate(2))


pool=[0, 1, 0, 1, 0, 0]
fitness=2
score=0.3333333333333333

CROSSOVER
from_left=True, crossover_index=1
lhs=[0, 1, 0, 1, 0, 0],rhs=[0, 0, 1, 1, 0, 1] -> lhs=[0, 1, 0, 1, 0, 0],rhs=[0, 0, 1, 1, 0, 1].
from_left=False, crossover_index=6
lhs=[0, 1, 0, 1, 0, 0],rhs=[0, 0, 1, 1, 0, 1] -> lhs=[0, 1, 0, 1, 0, 0],rhs=[0, 0, 1, 1, 0, 1].
from_left=True, crossover_index=5
lhs=[0, 1, 0, 1, 0, 0],rhs=[0, 0, 1, 1, 0, 1] -> lhs=[0, 0, 1, 1, 0, 0],rhs=[0, 1, 0, 1, 0, 1].
None

MUTATION
indices=[2, 1], pool=[0, 0, 1, 1, 0, 0] -> pool=[0, 1, 0, 1, 0, 0]
None


In [382]:
class GeneticAlgorithm:
    def __init__(self, **kwargs):
        self._initialize(**kwargs)
        self._chromosomes = []
    
    
    def _initialize(self, **kwargs):
        self._chromosomes_no = int(kwargs["chromosomes_no"])
        self._genes_no = int(kwargs["genes_no"])
        self._generations_no = int(kwargs["generations_no"])
        self._crossover_points = int(kwargs["crossover_points"])
        self._mutation_points = int(kwargs["mutation_points"])
        self._crossover_probability = float(kwargs["crossover_probability"])
        self._mutation_probability = float(kwargs["mutation_probability"])
    
    
    def _generate_initial_population(self):
        for i in range(0, self._chromosomes_no):
            self._chromosomes.append(Chromosome(self._genes_no))
    
    
    def _select_using_roulette(self):
        scores_sum, normalized = self._get_normalized_scores()
        self._sort_population(normalized)
        accumulated_normalized_fitness = self._get_accumulated_normalized_fitness(scores_sum)
        selection_indices = self._select_the_fittest(accumulated_normalized_fitness)
        #print('scores_sum=' + str(scores_sum))
        #print('normalized=' + str(normalized))
        #print('accumulated_normalized_fitness=' + str(accumulated_normalized_fitness))
        return selection_indices
        
                
    def _get_normalized_scores(self):
        scores_sum = 0.0
        normalized = []
        
        for i in range(0, len(self._chromosomes)):
            chromo = self._chromosomes[i]
            scores_sum += chromo.get_score()
            
        for j in range(0, len(self._chromosomes)):
            chromo = self._chromosomes[j]
            normalized.append(chromo.get_score() / scores_sum)
            
        return scores_sum, normalized
    
    
    def _get_accumulated_normalized_fitness(self, scores_sum):
        probability_sum = 0.0
        accumulated_fitness = []
        accumulated_fitness_sum = 0.0
        accumulated_normalized_fitness = []
        
        for i in range(0, len(self._chromosomes)):
            chromo = self._chromosomes[i]
            probability = probability_sum + (chromo.get_score() / scores_sum)
            probability_sum += probability
            accumulated_fitness.append(probability)
            
        for j in range(0, len(accumulated_fitness)):
            accumulated_fitness_sum += accumulated_fitness[j]
            
        for k in range(0, len(accumulated_fitness)):
            value = accumulated_fitness[k] / accumulated_fitness_sum
            accumulated_normalized_fitness.append(value)
            
        return accumulated_normalized_fitness
        
        
    def _sort_population(self, normalized):
        for i in range(0, len(normalized)):
            for j in range(0, len(normalized) - 1):
                previous_fitness = normalized[j]
                next_fitness = normalized[j+1]
                
                if previous_fitness > next_fitness:
                    self._swap_elements(normalized, j, j+1)
                    self._swap_elements(self._chromosomes, j, j+1)
                    
                    
    def _swap_elements(self, struct, previous_index, next_index):
        tmp = struct[previous_index]
        struct[previous_index] = struct[next_index]
        struct[next_index] = tmp
            
    
    def _select_the_fittest(self, accumulated_normalized_fitness):
        indices = []
        
        for i in range(0, len(self._chromosomes)):
            if rnd.random() > accumulated_normalized_fitness[i]:
                if not self._is_in(indices, i):
                    indices.append(i)
        
        return indices
    
    
    def _is_in(self, collection, element):
        for i in range(0, len(collection)):
            if collection[i] == element:
                return True
        
        return False
    
    
    def perform(self):
        print("START")
        self._generate_initial_population()
        iterations = 0
        
        while iterations < self._generations_no:
            selection_indices = self._select_using_roulette()
            offspring = self._create_offspring(selection_indices)
            iterations += 1
            
            #print('selection_indices=' + str(selection_indices))
            #print('offspring=', end='')
            #for i in range(0, len(offspring)):
            #    print(offspring[i].to_string(), end='')            
            #print('\n\n')
            
        print("END")

        
    def _create_offspring(self, selection_indices):
        offspring = []
        result_offspring = []
        population_size = 0
        
        while population_size < len(self._chromosomes):
            # Crossover first n combinations of selected individuals
            for i in range(0, len(selection_indices)):
                j = i
                lhs_chromo = self._chromosomes[selection_indices[i]]

                for j in range(0, len(selection_indices)):
                    child = Chromosome(lhs_chromo.size())
                    
                    if rnd.random() > self._crossover_probability:
                        rhs_chromo = self._chromosomes[selection_indices[j]]
                        child = lhs_chromo.get_copy()
                        child.crossover_with(rhs_chromo, self._crossover_points)
                        population_size += 1
                        
                    if rnd.random() > self._mutation_probability:
                        child.mutate(self._mutation_points)
                        
                    offspring.append(child)
        
        for i in range(0, len(self._chromosomes)):
            result_offspring.append(offspring[i])
        
        return result_offspring
    

In [381]:
#GeneticAlgorithm demo

params = {
    "chromosomes_no" : 5,                # population size
    "genes_no" : 7,                      # chromosome length
    "generations_no" : 3,                # number of iterations (converging steps)
    "crossover_points" : 2,
    "mutation_points" : 3,
    "crossover_probability" : 0.5,
    "mutation_probability" : 0.25
}

ga = GeneticAlgorithm(**params)
ga.perform()


START
scores_sum=2.2857142857142856
normalized=[0.0625, 0.1875, 0.25, 0.25, 0.25]
accumulated_normalized_fitness=[0.014705882352941176, 0.058823529411764705, 0.1323529411764706, 0.2647058823529412, 0.5294117647058824]
selection_indices=[0, 1, 2, 3, 4]
offspring=[[0, 1, 1, 0, 1, 1, 0],57%][[0, 1, 0, 0, 0, 0, 1],28%][[0, 0, 0, 0, 0, 1, 1],28%][[0, 1, 1, 0, 0, 1, 0],42%][[1, 1, 0, 1, 0, 0, 0],42%]


scores_sum=1.5714285714285712
normalized=[0.18181818181818185, 0.18181818181818185, 0.18181818181818185, 0.18181818181818185, 0.27272727272727276]
accumulated_normalized_fitness=[0.031746031746031744, 0.06349206349206349, 0.12698412698412698, 0.25396825396825395, 0.5238095238095238]
selection_indices=[0, 1, 2]
offspring=[[1, 1, 1, 1, 1, 1, 1],100%][[0, 1, 1, 0, 1, 0, 1],57%][[1, 1, 0, 0, 0, 1, 0],42%][[1, 0, 1, 1, 0, 0, 0],42%][[1, 0, 0, 1, 1, 1, 0],57%]


scores_sum=1.7142857142857142
normalized=[0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.25, 0.25]
accumulated_normalized

In [418]:
params = {
    "chromosomes_no" : 25,
    "genes_no" : 25,
    "generations_no" : 1,
    "crossover_points" : 2,
    "mutation_points" : 11,
    "crossover_probability" : 0.5,
    "mutation_probability" : 0.5
}

ga = GeneticAlgorithm(**params)
ga.perform()

for i in range(0, len(ga._chromosomes)):
    print(ga._chromosomes[i].to_string())
    

START
END
[[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],40%]
[[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],40%]
[[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],40%]
[[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],36%]
[[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],40%]
[[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],40%]
[[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],44%]
[[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],40%]
[[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],40%]
[[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],36%]
[[1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0],40%]
[[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],40%]
[[1, 1

In [232]:
print()
